In [ ]:
#Data: A  Nationa survey of school nurses collected December 2021- January 2022. This was a critical time period of the COVID-19 pandemic, months after the COVID-19 vaccine was approved for school-aged children 5-11 years old.
# 1,536 school nurses in the US responded to the survey, 289 filled out either one or both of the open-ended text option questions. 


In [ ]:
#. Purpose of Wrangle (i.e. original desired end state)
#The purpose of this wrangle was to focus on the two open-ended text responses using python code, such as Regex, learned in this class.
# I used the information I have from a scoping review of the literature I conducted on parental vaccine hesitancy towards the Covid-19 vaccine for theri children 5-11 years old and see if the school nurses texts had similar themes.
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib notebook
import re
import pprint



In [ ]:
#2. Sample of input data (this can be a screenshot, text output, etc.)
import pandas as pd

# Read the CSV file into a DataFrame
pdf = pd.read_csv('C:/Users/chl_b/dw_work/data/Open_responses_NASN_22.csv')

# Filter rows where the text response is empty(NaN) or a variation of NO or NA


filtered_pdf = pdf[(pdf['Parent_Questions'] != 'No') & (pdf['Parent_Questions'].notna()) & (pdf['Other_SN_Questions'] != 'No') & (pdf['Other_SN_Questions'].notna())]

# Display the first six rows for each column
print("First six rows for Parent_Questions:")
print(filtered_pdf['Parent_Questions'].head(6))

print("\nFirst six rows for Other_SN_Questions:")
print(filtered_pdf['Other_SN_Questions'].head(6))


In [ ]:
#3. Strategy and libraries utilized: List comprehension, locate (loc), RegEx
import numpy as np
import pandas as pd
import pylab as plt
%matplotlib notebook
import re
import pprint

# Using list comprehension, I want to describe how many responded to each question without saying no answers
# Define variations of "NO"

# Define variations of "NO"
no_variations = ['no', 'n/a', 'na']

# Filter rows for pq where the text response is not a variation of 'No' or empty
filtered_pq = pdf[(~pdf['Parent_Questions'].str.lower().isin(no_variations)) & (pdf['Parent_Questions'].notna())]

# Filter rows for snq where the text response is not a variation of 'No' or empty
filtered_snq = pdf[(~pdf['Other_SN_Questions'].str.lower().isin(no_variations)) & (pdf['Other_SN_Questions'].notna())]

# Count the total number of responses for each column
total_responses_pq = filtered_pq.shape[0]  # Shape[0] gives the number of rows
total_responses_snq = filtered_snq.shape[0]  # Shape[0] gives the number of rows

print("Total number of responses for'Parent_Questions'  :", total_responses_pq)
print("Total number of responses for 'Other_SN_Questions':", total_responses_snq)


In [ ]:
#Strategy using .Loc and RegEx to find strings that contain words that correlate to themes I found in my scoping review.
# Variations of words such as 'politics', 'effects', fertility', 'vaccin*', 'distress'
import pandas as pd
import re

# Function to search for the word 'politic' and its variations, handling non-string types
def find_politics(text):
    if isinstance(text, str):  # Check if the input is a string
        pattern = r'\bpolitic\w*'  # Regular expression pattern to find words starting with 'politic'
        return bool(re.search(pattern, text, re.IGNORECASE))
    else:
        return False  # Return False if the input is not a string

# Apply the function to both columns
pdf['mentions_politics_in_snq'] = pdf['Other_SN_Questions'].apply(find_politics)
pdf['mentions_politics_in_pq'] = pdf['Parent_Questions'].apply(find_politics)

# Filter rows where 'politic' was mentioned in either column
political_mentions = pdf[(pdf['mentions_politics_in_snq']) | (pdf['mentions_politics_in_pq'])]

# Check if the DataFrame is empty and print responses
if not political_mentions.empty:
    # Print responses from 'Other SN Questions'
    print("Responses from 'Other_SN_Questions' where 'politic' was mentioned:")
    for response in political_mentions.loc[political_mentions['mentions_politics_in_snq'], 'Other_SN_Questions']:
        print(response)
    
    # Print responses from 'Parent Questions'
    print("\nResponses from 'Parent_Questions' where 'politic' was mentioned:")
    for response in political_mentions.loc[political_mentions['mentions_politics_in_pq'], 'Parent_Questions']:
        print(response)
else:
    print("No participants found with the specified criteria.")



In [ ]:
#4. Problems, hiccups, rabbit holes, etc., with suggestions to avoid
# I did initial work to create the demographics of the participants that I would need to include in a publication, Table 1 descriptive statistics.
#I do not recommend using python for this as I can get nicely formatted tables for nursing journals in APA7 through using R. 
# I also learned the importance of missing values and NaNs, depending on what your datafile and what you are doing in your wranglling process. 
#However, python using RegEx worked wonderfully for doing the text mining needed for this wrangle.
# I also was reminded many time of importance of naming conventions and keeping original dataset safe somewhere as I had to double check when wrangling for unintended consequences of my hiccups in coding

In [ ]:
#visualizations for text mining using the dataframe 'pdf'. 
#I looked into Natural languae toolkit (NLTK)-a tool for teaching, and working in, computational linguistics using Python,” and “an amazing library to play with natural language.”
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk

# Download the set of stop words the first time
nltk.download('punkt')
nltk.download('stopwords')

#Define function to process text

def process_text(texts):
    stop_words = set(stopwords.words('english'))  # English stop words
    word_counts = Counter()

    for text in texts:
        if isinstance(text, str):
            words = word_tokenize(text.lower())  # Tokenize and convert to lower case
            filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
            word_counts.update(filtered_words)
    
    return word_counts


In [ ]:
# Apply the function and prepare data for visualization
# Combine text from 'Other SN Questions' and 'Parent Questions'
all_texts = pd.concat([pdf['Other_SN_Questions'].dropna(), pdf['Parent_Questions'].dropna()])

# Get word counts
word_counts = process_text(all_texts)

# Convert to a DataFrame
word_freq = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])  # Top 20 words


In [ ]:
# visualize the data
plt.figure(figsize=(10, 6))
sns.barplot(x='Frequency', y='Word', data=word_freq, palette='viridis')
plt.title('Top 20 Most Frequent Words in Responses')
plt.xlabel('Frequency')
plt.ylabel('Word')
plt.show()


In [ ]:
# make this more interactive
!pip install plotly
import plotly.express as px

# Assuming 'word_freq' is your DataFrame
fig = px.bar(word_freq, y='Word', x='Frequency', orientation='h',
             title='Top 20 Most Frequent Words in Responses',
             labels={'Frequency': 'Frequency', 'Word': 'Word'},
             hover_data={'Word': True, 'Frequency': True})  # Enables hovering details

fig.update_layout(
    xaxis_title='Frequency',
    yaxis_title='Word',
    yaxis={'categoryorder':'total ascending'}  # This makes the highest values appear on top
)
fig.show()


In [ ]:
# I can use nldk for sentiment analysis and topic modeling
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def analyze_sentiment(texts):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = [sia.polarity_scores(text) for text in texts if isinstance(text, str)]
    return pd.DataFrame(sentiment_scores)

# Example usage
sentiment_scores = analyze_sentiment(pdf['Other_SN_Questions'].dropna())
print(sentiment_scores.head())

sentiment_scores = analyze_sentiment(pdf['Parent_Questions'].dropna())
print(sentiment_scores.head())


In [ ]:
# So how do I interpret this?
import pandas as pd
import numpy as np

# Sample data
data_1 = {
    "neg": [0.000, 0.053, 1.000, 0.000, 1.000],
    "neu": [0.878, 0.833, 0.000, 1.000, 0.000],
    "pos": [0.122, 0.114, 0.000, 0.000, 0.000],
    "compound": [0.3612, 0.5202, -0.2960, 0.0000, -0.2960]
}

data_2 = {
    "neg": [0.000, 0.262, 0.000, 0.153, 0.000],
    "neu": [1.000, 0.738, 1.000, 0.847, 1.000],
    "pos": [0.0, 0.0, 0.0, 0.0, 0.0],
    "compound": [0.0000, -0.4939, 0.0000, -0.3447, 0.0000]
}

df1 = pd.DataFrame(data_1)
df2 = pd.DataFrame(data_2)

# Calculating averages
average_scores_df1 = df1.mean()
average_scores_df2 = df2.mean()

# Display the averages
print("Average Scores for SN Questions Set:")
print(average_scores_df1)
print("\nAverage Scores for Parent Questions Set:")
print(average_scores_df2)


In [ ]:
# and plot this for visualization
import matplotlib.pyplot as plt


# Plotting the histograms
plt.figure(figsize=(10, 6))
plt.hist(df1['compound'], bins=10, color='blue', alpha=0.7, label='SN  Question')
plt.hist(df2['compound'], bins=10, color='green', alpha=0.7, label='Parent Questions')

plt.title('Distribution of Compound Sentiment Scores for Each Question')
plt.xlabel('Compound Score')
plt.ylabel('Frequency')
plt.legend(loc='upper right')  # Adds a legend to specify which color represents which question
plt.grid(True)
plt.show()

# this shows that the sentiment from the parents questions was much more negative sentiment than from the sn questions.


In [ ]:
#Topic modeling

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def perform_lda(texts, num_topics=5):
    vectorizer = CountVectorizer(stop_words='english')
    text_data = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
    lda.fit(text_data)
    return lda, vectorizer

# Example usage
texts = pdf['Other_SN_Questions'].dropna().tolist()
lda_model, vectorizer = perform_lda(texts)
print(lda_model.components_)  # Display the topics

# Example usage
texts = pdf['Parent_Questions'].dropna().tolist()
lda_model, vectorizer = perform_lda(texts)
print(lda_model.components_)  # Display the topics


In [ ]:
# But what does this mean???
feature_names = vectorizer.get_feature_names_out()

# For each topic, find the words with the highest probability
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

print_top_words(lda_model, feature_names, 10)


In [ ]:
#6. A brief description of "what comes next"
# I have alot of the LIkert scale questions and demographics I can use to do statistical regression analysis with and I am working with
# a collegue at the college of nursing to write this up. I did do some prelimary work 
# in 2022 and created a poster so I will look at that data and make sure it was cleaned up and wrnagled properly.
#Thank you!

In [ ]:
# and last visualize the topics. pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

#The visualization is intended to be used within an IPython notebook but can also be saved to a stand-alone HTML file for easy sharing.https://pypi.org/project/pyLDAvis/
# I have to figure out why this is not working!
!pip uninstall pyLDAvis
!pip install pyLDAvis


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
import pyLDAvis

# Assuming 'lda_model' is your trained LDA model and 'text_data' is a document-term matrix
panel = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_ / lda_model.components_.sum(axis=1)[:, np.newaxis],  # Normalized topic-term distributions
    doc_topic_dists=lda_model.transform(text_data),  # Document-topic distributions
    doc_lengths=text_data.sum(axis=1).A1,  # Total number of words per document
    vocab=vectorizer.get_feature_names_out(),  # Vocabulary
    term_frequency=text_data.sum(axis=0).A1  # Total word counts across all documents
)

pyLDAvis.display(panel)


